In [ ]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk

WIDTH, HEIGHT = 800, 600

class CameraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Camera Viewer")
        self.root.geometry(f"{WIDTH}x{HEIGHT}")

        self.cap = None
        self.running = False

        self.label = tk.Label(root)
        self.label.pack(expand=True)

        controls = tk.Frame(root)
        controls.pack(pady=10)

        tk.Button(controls, text="Start", command=self.start_camera).pack(side="left", padx=10)
        tk.Button(controls, text="Stop", command=self.stop_camera).pack(side="left", padx=10)

        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    def start_camera(self):
        if self.running:
            return  # already running

        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Could not open camera.")
            return

        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

        self.running = True
        self.update_frame()

    def update_frame(self):
        if not self.running or self.cap is None:
            return

        ret, frame = self.cap.read()
        if ret:
            frame = cv2.flip(frame, 1)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)

            self.label.imgtk = imgtk
            self.label.configure(image=imgtk)

        self.root.after(15, self.update_frame)

    def stop_camera(self):
        self.running = False

        if self.cap:
            self.cap.release()
            self.cap = None

        self.label.configure(image="")
        self.label.imgtk = None

    def on_close(self):
        self.stop_camera()
        self.root.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    app = CameraApp(root)
    root.mainloop()